In [1]:
def train_CNN(train_directory,target_size=(200,200), classes=None,
              batch_size=128,num_epochs=20,num_classes=5,verbose=0):
    """
    Trains a conv net for the flowers dataset with a 5-class classifiction output
    Also provides suitable arguments for extending it to other similar apps
    
    Arguments:
            train_directory: The directory where the training images are stored in separate folders.
                            These folders should be named as per the classes.
            target_size: Target size for the training images. A tuple e.g. (200,200)
            classes: A Python list with the classes 
            batch_size: Batch size for training
            num_epochs: Number of epochs for training
            num_classes: Number of output classes to consider
            verbose: Verbosity level of the training, passed on to the `fit_generator` method
    Returns:
            A trained conv net model
    
    """
    from keras.preprocessing.image import ImageDataGenerator
    import tensorflow as tf
    from keras.optimizers import RMSprop
    
    # ImageDataGenerator object instance with scaling
    train_datagen = ImageDataGenerator(rescale=1/255)

    # Flow training images in batches using the generator
    train_generator = train_datagen.flow_from_directory(
            train_directory,  # This is the source directory for training images
            target_size=target_size,  # All images will be resized to 200 x 200
            batch_size=batch_size,
            # Specify the classes explicitly
            classes = classes,
            # Since we use categorical_crossentropy loss, we need categorical labels
            class_mode='categorical')
    
    input_shape = tuple(list(target_size)+[3])
    
    # Model architecture
    model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 200x 200 with 3 bytes color
    # The first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a dense layer
    tf.keras.layers.Flatten(),
    # 512 neuron in the fully-connected layer
    tf.keras.layers.Dense(512, activation='relu'),
    # 5 output neurons for 5 classes with the softmax activation
    tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    
    # Optimizer and compilation
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    # Total sample count
    total_sample=train_generator.n
    
    # Training
    model.fit_generator(
        train_generator, 
        steps_per_epoch=int(total_sample/batch_size),  
        epochs=num_epochs,
        verbose=verbose)
    
    return model
train_directory = "stone_new"
trained_model=train_CNN(train_directory=train_directory,classes=['mm_10','mm_20','mm_265','sand_plast','sand_top'],
                        num_epochs=25,num_classes=5,verbose=1)


Using TensorFlow backend.


Found 6806 images belonging to 5 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 53 steps
Epoch 1/25
53/53 [==============================] - 322s 6s/step - loss: 0.9302 - accuracy: 0.5684
Epoch 2/25
53/53 [==============================] - 270s 5s/step - loss: 0.5141 - accuracy: 0.7800
Epoch 3/25
53/53 [==============================] - 250s 5s/step - loss: 0.3728 - accuracy: 0.8362
Epoch 4/25
53/53 [==============================] - 250s 5s/step - loss: 0.3677 - accuracy: 0.8408
Epoch 5/25
53/53 [==============================] - 245s 5s/step - loss: 0.2998 - accuracy: 0.8730
Epoch 6/25
53/53 [==============================] - 251s 5s/step - loss: 0.2719 - accuracy: 0.8871
Epoch 7/25
53/53 [==============================] - 269s 5s/step - loss: 0.2413 - accuracy: 0.8983
Epoch 8/25
53/53 [==============================] - 258s 5s/step - loss: 0.2038 - accuracy: 0.9220
Epoch 9/25
53/53 [============================

In [1]:
from PIL import Image
import numpy as np
test_image=Image.open('flower-test/rose2.jpg')
#img_cup=Image.open('../Data/101_ObjectCategories/Test_images/cup-1.jpg')
#test_image_cup=Image.open('../Data/101_ObjectCategories/Test_images/crab-cup-1.jpg')
test_image = test_image.resize((200,200))
#img_cup = img_cup.resize((200,200))
#test_image_cup = test_image_cup.resize((200,200))
test_image=np.expand_dims(test_image,axis=0)
#img_cup=np.expand_dims(img_cup,axis=0)
#test_image_cup=np.expand_dims(test_image_cup,axis=0)
predict_class = trained_model.predict(test_image)
print(type(predict_class))


FileNotFoundError: [Errno 2] No such file or directory: 'flower-test/rose2.jpg'

In [2]:
trained_model.save('cnn_stone_new.h5')